In [1]:
import pandas as pd
#import numpy as 

In [2]:
# File path for first csv
schools_data= "schools_complete.csv"

# Read our csv into Pandas as a dataframe and display first 5 rows
schools_data_df = pd.read_csv(schools_data)
schools_data_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
# Rename "name" column to schools
schools_data_df = schools_data_df.rename(index=str, columns={"name": "school"})
schools_data_df.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
# Set new index to last_name instead of 0,1,2 etc
#new_schools_data_df = schools_data_df.set_index("name")
#new_schools_data_df.head()

In [5]:
# File path for second csv
student_data = "students_complete.csv"

# Read our csv into Pandas and display first 5 rows
student_data_df = pd.read_csv(student_data)
student_data_df.head()


,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [21]:
number_student_records = len(student_data_df)
print(number_student_records)

39170


In [6]:
# Get a list of all of our columns for reference, in first csv
schools_data_df.columns

Index(['School ID', 'school', 'type', 'size', 'budget'], dtype='object')

In [7]:
#Get a list of all of our columns for reference, in 2nd csv
student_data_df.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [8]:
# Merge Dataframes
merge_table_df = pd.merge(schools_data_df, student_data_df, on="school")
merge_table_df.head(20)

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
5,0,Huang High School,District,2917,1910635,5,Bryan Miranda,M,9th,94,94
6,0,Huang High School,District,2917,1910635,6,Sheena Carter,F,11th,82,80
7,0,Huang High School,District,2917,1910635,7,Nicole Baker,F,12th,96,69
8,0,Huang High School,District,2917,1910635,8,Michael Roth,M,10th,95,87
9,0,Huang High School,District,2917,1910635,9,Matthew Greene,M,10th,96,84


In [20]:
total_records = len(merge_table_df)
print(total_records)

39170


In [15]:
# Find total number of schools
number_of_schools = (merge_table_df["school"]).nunique()
print(number_of_schools)

15


In [10]:
# Find total count of number of students 
# Should be the total of the .unique values [2917 2949 1761 4635 1468 2283 1858 4976  427  962 1800 3999 4761 2739 1635]
number_students = (merge_table_df["size"]).sum()
print(number_students)

130551930


In [17]:
# Find the total budget 
budget = (merge_table_df["budget"]).sum()
print(budget)

82932329558


In [18]:
# Find the average math score
average_math_score = merge_table_df.math_score.mean()
print(average_math_score)

78.98537145774827


In [11]:
# Find the average reading score
average_reading_score = merge_table_df.reading_score.mean()
print(average_reading_score)

81.87784018381414


In [22]:
# Find the percent passing math
percent_passing_math = merge_table_df[merge_table_df["math_score"]> 60]
percent_passing_math = len(percent_passing_math)
percent_passing_math = (percent_passing_math/total_records) * 100
print(percent_passing_math)

90.90630584631096


In [ ]:
# Create new dataframe from calculations
district_summary = pd.DataFrame({"Total Schools": [number_of_schools],
                                   "Total Students": [number_students],
                                   "Total Budget": [budget],
                                   "Average Math Score": [average_age],
                                   "Average Reading Score": [average_reading_score],
                                   "Percent Passing":[female_percent],
                                   
})